In [ ]:
%%capture
!pip install pytorch-tabnet

In [ ]:
import numpy as np
import pandas as pd
import os
import shutil
import joblib
import pickle
import json

# ml frameworks
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.datasets import make_classification # for test of funcs
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef,
    precision_recall_curve,
    auc,
    RocCurveDisplay,
    PrecisionRecallDisplay,
)

import torch
#time management
from tqdm import tqdm
import time

#stats
from scipy.stats import wasserstein_distance
from scipy.spatial.distance import cdist
from itertools import combinations

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.callbacks import EarlyStopping

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [ ]:
X_train_csv = pd.read_csv("/content/drive/MyDrive/BC-V/CARVEME/50/X_train_BC-V-50.csv")
X_val_csv = pd.read_csv("/content/drive/MyDrive/BC-V/CARVEME/50/X_val_BC-V-50.csv")
X_test_csv = pd.read_csv("/content/drive/MyDrive/BC-V/CARVEME/50/X_test_BC-V-50.csv")
y_train_csv = pd.read_csv("/content/drive/MyDrive/BC-V/CARVEME/50/y_train_BC-V-50.csv")
y_val_csv = pd.read_csv("/content/drive/MyDrive/BC-V/CARVEME/50/y_val_BC-V-50.csv")
y_test_csv = pd.read_csv("/content/drive/MyDrive/BC-V/CARVEME/50/y_test_BC-V-50.csv")

In [ ]:
def to_npy(X_train_csv, X_val_csv, X_test_csv, y_train_csv, y_val_csv, y_test_csv):
    X_train = X_train_csv.to_numpy()
    X_val = X_val_csv.to_numpy()
    X_test = X_test_csv.to_numpy()
    y_train = y_train_csv.to_numpy()
    y_val = y_val_csv.to_numpy()
    y_test = y_test_csv.to_numpy()

    # Save the arrays as .npy files
    np.save("X_train.npy", X_train)
    np.save("X_val.npy", X_val)
    np.save("X_test.npy", X_test)
    np.save("y_train.npy", y_train)
    np.save("y_val.npy", y_val)
    np.save("y_test.npy", y_test)

    print("CSV onverted to .npy format.")

In [ ]:
to_npy(X_train_csv, X_val_csv, X_test_csv, y_train_csv, y_val_csv, y_test_csv)

CSV onverted to .npy format.


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = np.load("X_train.npy"), np.load("y_train.npy"), np.load("X_val.npy"), np.load("y_val.npy"), np.load("X_test.npy"), np.load("y_test.npy")

In [ ]:
y_train = y_train.reshape(-1)
y_val = y_val.reshape(-1,)
y_test = y_test.reshape(-1)

In [ ]:
def create_confusion_matrix(y_true, y_pred):
    '''
    Description: Create a confusion matrix.
    Arguments: y_true (array-like): Ground truth labels;
               y_pred (array-like): Predicted labels.
    Outputs:
        pd.DataFrame: A confusion matrix as a pandas DataFrame.
    '''
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=["True Negative", "True Positive"],
                             columns=["Predicted Negative", "Predicted Positive"])
    return cm_df


def score_metrics(y_true, y_pred, y_prob):
    '''
    Description: Calculate various metrics for binary classification.
    Arguments: y_true (array-like): Ground truth labels;
               y_pred (array-like): Predicted labels;
               y_prob (array-like): Predicted probabilities for the positive class.
    Outputs:
        dict
    '''
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "ROC AUC": roc_auc_score(y_true, y_prob),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "F1 Score": f1_score(y_true, y_pred),
        "MCC": matthews_corrcoef(y_true, y_pred),
    }
    # PR AUC
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    metrics["PR AUC"] = auc(recall, precision)
    return metrics


def train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test,
                             output_dir="tabnet_results", seed=4221,
                             max_epochs=100, patience=10):
    '''
    Description: Train and evaluate TabNet on binclass.

    Arguments:
        X_train, y_train: Training data
        X_val, y_val: Validation data
        X_test, y_test: Test data
        output_dir: Directory to save results
        seed: Random seed

    Outputs:
        json
    '''

    os.makedirs(output_dir, exist_ok=True)

    # Init
    clf = TabNetClassifier(
        cat_idxs=[],
        cat_dims=[],
        cat_emb_dim=1,
        optimizer_fn=torch.optim.AdamW,
        optimizer_params=dict(lr=1e-4, weight_decay=0.02),
        scheduler_params={"step_size":50, "gamma":0.99},
        scheduler_fn=torch.optim.lr_scheduler.StepLR,
        mask_type='entmax',
        n_d=64,
        n_a=64,
        seed=seed,
        device_name=device
    )

    # Train
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_metric=['accuracy'],
        max_epochs=max_epochs,
        patience=patience,
        batch_size=1024
    )

    # Test
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)[:, 1]

    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "ROC AUC": roc_auc_score(y_test, y_proba),
        "Recall": recall_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred),
        "MCC": matthews_corrcoef(y_test, y_pred)
    }

    with open(os.path.join(output_dir, "tabnet_metrics.json"), 'w') as f:
        json.dump(metrics, f, indent=4)

    clf.save_model(os.path.join(output_dir, "tabnet_model.zip"))

    print(f"\nTest Metrics:")
    print(f"Accuracy: {metrics['Accuracy']:.4f}")
    print(f"MCC: {metrics['MCC']:.4f}")

    return metrics

#AGORA-BC-I-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.92274 | val_0_accuracy: 0.61213 |  0:00:13s
epoch 1  | loss: 0.84115 | val_0_accuracy: 0.62834 |  0:00:31s
epoch 2  | loss: 0.78972 | val_0_accuracy: 0.6379  |  0:00:46s
epoch 3  | loss: 0.75737 | val_0_accuracy: 0.6498  |  0:01:00s
epoch 4  | loss: 0.73588 | val_0_accuracy: 0.65593 |  0:01:20s
epoch 5  | loss: 0.7184  | val_0_accuracy: 0.6628  |  0:01:42s
epoch 6  | loss: 0.70531 | val_0_accuracy: 0.66339 |  0:01:55s
epoch 7  | loss: 0.69527 | val_0_accuracy: 0.66624 |  0:02:13s
epoch 8  | loss: 0.6859  | val_0_accuracy: 0.6696  |  0:02:26s
epoch 9  | loss: 0.67728 | val_0_accuracy: 0.67173 |  0:02:39s
epoch 10 | loss: 0.67124 | val_0_accuracy: 0.67249 |  0:02:55s
epoch 11 | loss: 0.66769 | val_0_accuracy: 0.67252 |  0:03:14s
epoch 12 | loss: 0.66201 | val_0_accuracy: 0.67401 |  0:03:27s
epoch 13 | loss: 0.65843 | val_0_accuracy: 0.6737  |  0:03:40s
epoch 14 | loss: 0.65426 | val_0_accuracy: 0.6751  |  0:03:54s
epoch 15 | loss: 0.65149 | val_0_accuracy: 0.67564 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.7117
MCC: 0.2540


{'Accuracy': 0.7116613296152726,
 'Precision': 0.7171460461921525,
 'Recall': 0.9467921106325097,
 'F1': 0.8161219404953832,
 'MCC': np.float64(0.25396405497208674)}

#AGORA-BC-II-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.4141  | val_0_accuracy: 0.60191 |  0:00:24s
epoch 1  | loss: 0.86738 | val_0_accuracy: 0.62229 |  0:00:48s
epoch 2  | loss: 0.81039 | val_0_accuracy: 0.62249 |  0:01:14s
epoch 3  | loss: 0.77645 | val_0_accuracy: 0.62677 |  0:01:37s
epoch 4  | loss: 0.7534  | val_0_accuracy: 0.62747 |  0:02:01s
epoch 5  | loss: 0.73656 | val_0_accuracy: 0.6317  |  0:02:25s
epoch 6  | loss: 0.72383 | val_0_accuracy: 0.62888 |  0:02:49s
epoch 7  | loss: 0.71216 | val_0_accuracy: 0.63458 |  0:03:13s
epoch 8  | loss: 0.70168 | val_0_accuracy: 0.63724 |  0:03:36s
epoch 9  | loss: 0.69406 | val_0_accuracy: 0.63866 |  0:04:00s
epoch 10 | loss: 0.6868  | val_0_accuracy: 0.64071 |  0:04:24s
epoch 11 | loss: 0.68198 | val_0_accuracy: 0.64063 |  0:04:48s
epoch 12 | loss: 0.67626 | val_0_accuracy: 0.64256 |  0:05:12s
epoch 13 | loss: 0.6727  | val_0_accuracy: 0.64451 |  0:05:35s
epoch 14 | loss: 0.66922 | val_0_accuracy: 0.64481 |  0:05:58s
epoch 15 | loss: 0.66567 | val_0_accuracy: 0.64507 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


#AGORA-BC-III-100

In [ ]:
y_train[y_train == 2] = 1
y_test[y_test == 2] = 1
y_val[y_val == 2] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.68684 | val_0_accuracy: 0.71839 |  0:00:16s
epoch 1  | loss: 0.72137 | val_0_accuracy: 0.77878 |  0:00:33s
epoch 2  | loss: 0.64308 | val_0_accuracy: 0.78332 |  0:00:49s
epoch 3  | loss: 0.61904 | val_0_accuracy: 0.78417 |  0:01:06s
epoch 4  | loss: 0.59999 | val_0_accuracy: 0.78478 |  0:01:22s
epoch 5  | loss: 0.58788 | val_0_accuracy: 0.7875  |  0:01:38s
epoch 6  | loss: 0.57557 | val_0_accuracy: 0.78819 |  0:01:55s
epoch 7  | loss: 0.56791 | val_0_accuracy: 0.78873 |  0:02:11s
epoch 8  | loss: 0.56098 | val_0_accuracy: 0.78942 |  0:02:28s
epoch 9  | loss: 0.5538  | val_0_accuracy: 0.78999 |  0:02:45s
epoch 10 | loss: 0.54967 | val_0_accuracy: 0.79052 |  0:03:01s
epoch 11 | loss: 0.54563 | val_0_accuracy: 0.79067 |  0:03:17s
epoch 12 | loss: 0.5398  | val_0_accuracy: 0.79083 |  0:03:33s
epoch 13 | loss: 0.53646 | val_0_accuracy: 0.79092 |  0:03:50s
epoch 14 | loss: 0.53261 | val_0_accuracy: 0.79109 |  0:04:06s
epoch 15 | loss: 0.52947 | val_0_accuracy: 0.79113 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.8223
MCC: 0.3597


{'Accuracy': 0.8223149420773611,
 'Precision': 0.8422803783176428,
 'ROC AUC': np.float64(0.7894366501189221),
 'Recall': 0.954967100380943,
 'F1': 0.8950910323960561,
 'MCC': np.float64(0.3596893208475966)}

#AGORA-BC-IV-100

In [ ]:
y_train[y_train == 1] = 0
y_train[y_train == 3] = 1
y_test[y_test == 1] = 0
y_test[y_test == 3] = 1
y_val[y_val == 1] = 0
y_val[y_val == 3] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.12131 | val_0_accuracy: 0.50515 |  0:00:12s
epoch 1  | loss: 0.93003 | val_0_accuracy: 0.52636 |  0:00:24s
epoch 2  | loss: 0.86949 | val_0_accuracy: 0.55486 |  0:00:37s
epoch 3  | loss: 0.82692 | val_0_accuracy: 0.58697 |  0:00:49s
epoch 4  | loss: 0.79477 | val_0_accuracy: 0.60279 |  0:01:02s
epoch 5  | loss: 0.7727  | val_0_accuracy: 0.61577 |  0:01:14s
epoch 6  | loss: 0.75589 | val_0_accuracy: 0.62465 |  0:01:27s
epoch 7  | loss: 0.74319 | val_0_accuracy: 0.62936 |  0:01:40s
epoch 8  | loss: 0.73078 | val_0_accuracy: 0.63565 |  0:01:52s
epoch 9  | loss: 0.72203 | val_0_accuracy: 0.63678 |  0:02:04s
epoch 10 | loss: 0.71672 | val_0_accuracy: 0.6403  |  0:02:17s
epoch 11 | loss: 0.70753 | val_0_accuracy: 0.63939 |  0:02:29s
epoch 12 | loss: 0.70197 | val_0_accuracy: 0.6429  |  0:02:41s
epoch 13 | loss: 0.69592 | val_0_accuracy: 0.6439  |  0:02:53s
epoch 14 | loss: 0.69203 | val_0_accuracy: 0.64608 |  0:03:05s
epoch 15 | loss: 0.68771 | val_0_accuracy: 0.64495 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.6505
MCC: 0.0366


{'Accuracy': 0.6504970354856789,
 'Precision': 0.47783251231527096,
 'ROC AUC': np.float64(0.5395157713892924),
 'Recall': 0.024616165461235885,
 'F1': 0.04682032098467479,
 'MCC': np.float64(0.03664662191984017)}

#AGORA-BC-V-100

In [ ]:
y_train[y_train == 3] = 1
y_test[y_test == 3] = 1
y_val[y_val == 3] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.6405  | val_0_accuracy: 0.49615 |  0:00:07s
epoch 1  | loss: 1.17257 | val_0_accuracy: 0.50997 |  0:00:14s
epoch 2  | loss: 1.00723 | val_0_accuracy: 0.51693 |  0:00:21s
epoch 3  | loss: 0.94753 | val_0_accuracy: 0.51581 |  0:00:29s
epoch 4  | loss: 0.91739 | val_0_accuracy: 0.51554 |  0:00:36s
epoch 5  | loss: 0.89441 | val_0_accuracy: 0.51226 |  0:00:43s
epoch 6  | loss: 0.86958 | val_0_accuracy: 0.51474 |  0:00:50s
epoch 7  | loss: 0.85413 | val_0_accuracy: 0.51706 |  0:00:58s
epoch 8  | loss: 0.83889 | val_0_accuracy: 0.51583 |  0:01:05s
epoch 9  | loss: 0.82674 | val_0_accuracy: 0.5163  |  0:01:12s
epoch 10 | loss: 0.81357 | val_0_accuracy: 0.51944 |  0:01:19s
epoch 11 | loss: 0.80829 | val_0_accuracy: 0.51845 |  0:01:26s
epoch 12 | loss: 0.79961 | val_0_accuracy: 0.52051 |  0:01:34s
epoch 13 | loss: 0.79145 | val_0_accuracy: 0.5172  |  0:01:41s
epoch 14 | loss: 0.78549 | val_0_accuracy: 0.51653 |  0:01:48s
epoch 15 | loss: 0.77827 | val_0_accuracy: 0.52017 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.6029
MCC: 0.2006


{'Accuracy': 0.602889539954823,
 'Precision': 0.6198270661572491,
 'ROC AUC': np.float64(0.640596250073072),
 'Recall': 0.6498840396373603,
 'F1': 0.6344997941539728,
 'MCC': np.float64(0.20059601780734493)}

#AGORA-BC-I-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.60849 | val_0_accuracy: 0.38419 |  0:00:02s
epoch 1  | loss: 2.10933 | val_0_accuracy: 0.52201 |  0:00:05s
epoch 2  | loss: 1.66771 | val_0_accuracy: 0.65402 |  0:00:08s
epoch 3  | loss: 1.31062 | val_0_accuracy: 0.70536 |  0:00:11s
epoch 4  | loss: 1.0615  | val_0_accuracy: 0.74754 |  0:00:14s
epoch 5  | loss: 0.88271 | val_0_accuracy: 0.77469 |  0:00:17s
epoch 6  | loss: 0.78321 | val_0_accuracy: 0.78011 |  0:00:20s
epoch 7  | loss: 0.71571 | val_0_accuracy: 0.79156 |  0:00:23s
epoch 8  | loss: 0.68322 | val_0_accuracy: 0.80084 |  0:00:26s
epoch 9  | loss: 0.66615 | val_0_accuracy: 0.79983 |  0:00:29s
epoch 10 | loss: 0.64696 | val_0_accuracy: 0.80313 |  0:00:32s
epoch 11 | loss: 0.64241 | val_0_accuracy: 0.8052  |  0:00:35s
epoch 12 | loss: 0.637   | val_0_accuracy: 0.80715 |  0:00:38s
epoch 13 | loss: 0.61662 | val_0_accuracy: 0.80715 |  0:00:41s
epoch 14 | loss: 0.61978 | val_0_accuracy: 0.80648 |  0:00:44s
epoch 15 | loss: 0.61886 | val_0_accuracy: 0.80872 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.8067
MCC: 0.0012


{'Accuracy': 0.806714708675493,
 'Precision': 0.8094837261503928,
 'ROC AUC': np.float64(0.5208718332824486),
 'Recall': 0.9955141476880607,
 'F1': 0.8929124110182606,
 'MCC': np.float64(0.001198736198901935)}

#AGORA-BC-II-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.64581 | val_0_accuracy: 0.83429 |  0:00:19s
epoch 1  | loss: 0.57227 | val_0_accuracy: 0.85974 |  0:00:37s
epoch 2  | loss: 0.49882 | val_0_accuracy: 0.86272 |  0:00:55s
epoch 3  | loss: 0.48078 | val_0_accuracy: 0.86439 |  0:01:13s
epoch 4  | loss: 0.46992 | val_0_accuracy: 0.86409 |  0:01:32s
epoch 5  | loss: 0.46079 | val_0_accuracy: 0.86458 |  0:01:52s
epoch 6  | loss: 0.45005 | val_0_accuracy: 0.8649  |  0:02:10s
epoch 7  | loss: 0.4453  | val_0_accuracy: 0.86545 |  0:02:28s
epoch 8  | loss: 0.43831 | val_0_accuracy: 0.86543 |  0:02:46s
epoch 9  | loss: 0.43412 | val_0_accuracy: 0.86554 |  0:03:04s
epoch 10 | loss: 0.42878 | val_0_accuracy: 0.86563 |  0:03:23s
epoch 11 | loss: 0.42555 | val_0_accuracy: 0.86571 |  0:03:41s
epoch 12 | loss: 0.42207 | val_0_accuracy: 0.86619 |  0:03:59s
epoch 13 | loss: 0.41841 | val_0_accuracy: 0.86618 |  0:04:17s
epoch 14 | loss: 0.41647 | val_0_accuracy: 0.86625 |  0:04:34s
epoch 15 | loss: 0.41342 | val_0_accuracy: 0.86661 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.8670
MCC: 0.0005


{'Accuracy': 0.8669986750125646,
 'Precision': 0.14,
 'ROC AUC': np.float64(0.6909704001189755),
 'Recall': 0.00048212686824161444,
 'F1': 0.0009609444711373464,
 'MCC': np.float64(0.0004618365110498173)}

#AGORA-BC-III-50

In [ ]:
y_train[y_train == 2] = 1
y_test[y_test == 2] = 1
y_val[y_val == 2] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.85131 | val_0_accuracy: 0.90478 |  0:00:18s
epoch 1  | loss: 0.38065 | val_0_accuracy: 0.96276 |  0:00:35s
epoch 2  | loss: 0.20326 | val_0_accuracy: 0.96514 |  0:00:52s
epoch 3  | loss: 0.18986 | val_0_accuracy: 0.96566 |  0:01:09s
epoch 4  | loss: 0.18493 | val_0_accuracy: 0.96526 |  0:01:27s
epoch 5  | loss: 0.18105 | val_0_accuracy: 0.96563 |  0:01:44s
epoch 6  | loss: 0.17969 | val_0_accuracy: 0.96505 |  0:02:01s
epoch 7  | loss: 0.17812 | val_0_accuracy: 0.96517 |  0:02:18s
epoch 8  | loss: 0.17518 | val_0_accuracy: 0.96523 |  0:02:35s
epoch 9  | loss: 0.17243 | val_0_accuracy: 0.96564 |  0:02:53s
epoch 10 | loss: 0.17096 | val_0_accuracy: 0.96547 |  0:03:10s
epoch 11 | loss: 0.16952 | val_0_accuracy: 0.96531 |  0:03:28s
epoch 12 | loss: 0.16765 | val_0_accuracy: 0.96547 |  0:03:45s
epoch 13 | loss: 0.16799 | val_0_accuracy: 0.96506 |  0:04:02s

Early stopping occurred at epoch 13 with best_epoch = 3 and best_val_0_accuracy = 0.96566


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.9654
MCC: -0.0013


{'Accuracy': 0.9653989453753283,
 'Precision': 0.9661555702715093,
 'ROC AUC': np.float64(0.4974092135040594),
 'Recall': 0.9991887050890317,
 'F1': 0.982394530339523,
 'MCC': np.float64(-0.001336993002707583)}

#AGORA-BC-IV-50

In [ ]:
y_train[y_train == 1] = 0
y_train[y_train == 3] = 1
y_test[y_test == 1] = 0
y_test[y_test == 3] = 1
y_val[y_val == 1] = 0
y_val[y_val == 3] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.86237 | val_0_accuracy: 0.53994 |  0:00:07s
epoch 1  | loss: 1.27631 | val_0_accuracy: 0.58604 |  0:00:14s
epoch 2  | loss: 1.00918 | val_0_accuracy: 0.60476 |  0:00:21s
epoch 3  | loss: 0.92469 | val_0_accuracy: 0.61262 |  0:00:29s
epoch 4  | loss: 0.89249 | val_0_accuracy: 0.61136 |  0:00:36s
epoch 5  | loss: 0.86164 | val_0_accuracy: 0.61511 |  0:00:44s
epoch 6  | loss: 0.8443  | val_0_accuracy: 0.61428 |  0:00:51s
epoch 7  | loss: 0.82642 | val_0_accuracy: 0.61602 |  0:00:59s
epoch 8  | loss: 0.81193 | val_0_accuracy: 0.61788 |  0:01:06s
epoch 9  | loss: 0.80528 | val_0_accuracy: 0.61811 |  0:01:13s
epoch 10 | loss: 0.79142 | val_0_accuracy: 0.62048 |  0:01:21s
epoch 11 | loss: 0.78262 | val_0_accuracy: 0.61932 |  0:01:28s
epoch 12 | loss: 0.77495 | val_0_accuracy: 0.61932 |  0:01:35s
epoch 13 | loss: 0.76594 | val_0_accuracy: 0.62168 |  0:01:43s
epoch 14 | loss: 0.75876 | val_0_accuracy: 0.62116 |  0:01:51s
epoch 15 | loss: 0.75213 | val_0_accuracy: 0.62629 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.6378
MCC: 0.0636


{'Accuracy': 0.6377560017129758,
 'Precision': 0.6399234924655587,
 'ROC AUC': np.float64(0.614540291000591),
 'Recall': 0.9821874876026501,
 'F1': 0.7749467885313635,
 'MCC': np.float64(0.06355061938330761)}

#AGORA-BC-V-50

In [ ]:
y_train[y_train == 3] = 1
y_test[y_test == 3] = 1
y_val[y_val == 3] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.61872 | val_0_accuracy: 0.48851 |  0:00:05s
epoch 1  | loss: 1.80933 | val_0_accuracy: 0.70784 |  0:00:10s
epoch 2  | loss: 1.19997 | val_0_accuracy: 0.81025 |  0:00:16s
epoch 3  | loss: 0.8154  | val_0_accuracy: 0.84169 |  0:00:21s
epoch 4  | loss: 0.61617 | val_0_accuracy: 0.86012 |  0:00:27s
epoch 5  | loss: 0.53417 | val_0_accuracy: 0.86984 |  0:00:33s
epoch 6  | loss: 0.4938  | val_0_accuracy: 0.87264 |  0:00:38s
epoch 7  | loss: 0.4812  | val_0_accuracy: 0.87647 |  0:00:43s
epoch 8  | loss: 0.47439 | val_0_accuracy: 0.87517 |  0:00:48s
epoch 9  | loss: 0.46757 | val_0_accuracy: 0.8758  |  0:00:54s
epoch 10 | loss: 0.45889 | val_0_accuracy: 0.87686 |  0:00:59s
epoch 11 | loss: 0.45114 | val_0_accuracy: 0.87703 |  0:01:04s
epoch 12 | loss: 0.45248 | val_0_accuracy: 0.87679 |  0:01:09s
epoch 13 | loss: 0.44331 | val_0_accuracy: 0.87742 |  0:01:15s
epoch 14 | loss: 0.43584 | val_0_accuracy: 0.87787 |  0:01:20s
epoch 15 | loss: 0.43599 | val_0_accuracy: 0.87854 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.8795
MCC: 0.0295


{'Accuracy': 0.8794567658618754,
 'Precision': 0.881327888474266,
 'ROC AUC': np.float64(0.5347439154196365),
 'Recall': 0.9974500976134507,
 'F1': 0.9358003924866835,
 'MCC': np.float64(0.029474922870519598)}

#CARVEME-BC-I-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.97706 | val_0_accuracy: 0.58212 |  0:00:12s
epoch 1  | loss: 0.88992 | val_0_accuracy: 0.57444 |  0:00:24s
epoch 2  | loss: 0.83952 | val_0_accuracy: 0.5827  |  0:00:37s
epoch 3  | loss: 0.80456 | val_0_accuracy: 0.58692 |  0:00:49s
epoch 4  | loss: 0.77859 | val_0_accuracy: 0.597   |  0:01:01s
epoch 5  | loss: 0.75854 | val_0_accuracy: 0.59595 |  0:01:14s
epoch 6  | loss: 0.74519 | val_0_accuracy: 0.60119 |  0:01:26s
epoch 7  | loss: 0.73352 | val_0_accuracy: 0.60196 |  0:01:38s
epoch 8  | loss: 0.72573 | val_0_accuracy: 0.6047  |  0:01:51s
epoch 9  | loss: 0.71669 | val_0_accuracy: 0.60741 |  0:02:03s
epoch 10 | loss: 0.70936 | val_0_accuracy: 0.60816 |  0:02:15s
epoch 11 | loss: 0.7043  | val_0_accuracy: 0.6084  |  0:02:28s
epoch 12 | loss: 0.69881 | val_0_accuracy: 0.6122  |  0:02:40s
epoch 13 | loss: 0.6963  | val_0_accuracy: 0.61332 |  0:02:52s
epoch 14 | loss: 0.69098 | val_0_accuracy: 0.61422 |  0:03:05s
epoch 15 | loss: 0.6877  | val_0_accuracy: 0.61522 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.6409
MCC: 0.1819


{'Accuracy': 0.6408812398630384,
 'Precision': 0.5739928860947969,
 'ROC AUC': np.float64(0.6637471614317911),
 'Recall': 0.2699789899618707,
 'F1': 0.36723029292688736,
 'MCC': np.float64(0.18188677319872407)}

#CARVEME-BC-II-100

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.89359 | val_0_accuracy: 0.63627 |  0:00:14s
epoch 1  | loss: 0.81029 | val_0_accuracy: 0.63607 |  0:00:29s
epoch 2  | loss: 0.76629 | val_0_accuracy: 0.64268 |  0:00:43s
epoch 3  | loss: 0.73775 | val_0_accuracy: 0.65308 |  0:00:59s
epoch 4  | loss: 0.71502 | val_0_accuracy: 0.65536 |  0:01:14s
epoch 5  | loss: 0.69972 | val_0_accuracy: 0.66412 |  0:01:28s
epoch 6  | loss: 0.68776 | val_0_accuracy: 0.66314 |  0:01:43s
epoch 7  | loss: 0.67955 | val_0_accuracy: 0.66723 |  0:01:57s
epoch 8  | loss: 0.67074 | val_0_accuracy: 0.66778 |  0:02:12s
epoch 9  | loss: 0.66403 | val_0_accuracy: 0.67211 |  0:02:26s
epoch 10 | loss: 0.6591  | val_0_accuracy: 0.6726  |  0:02:41s
epoch 11 | loss: 0.65467 | val_0_accuracy: 0.67468 |  0:02:56s
epoch 12 | loss: 0.65081 | val_0_accuracy: 0.67666 |  0:03:10s
epoch 13 | loss: 0.64802 | val_0_accuracy: 0.67762 |  0:03:25s
epoch 14 | loss: 0.64494 | val_0_accuracy: 0.67755 |  0:03:40s
epoch 15 | loss: 0.64223 | val_0_accuracy: 0.67791 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.6827
MCC: 0.0051


{'Accuracy': 0.6827056572150442,
 'Precision': 0.3645833333333333,
 'ROC AUC': np.float64(0.5913421027924252),
 'Recall': 0.002765049770895876,
 'F1': 0.005488474204171241,
 'MCC': np.float64(0.005056431648396121)}

#CARVEME-BC-III-100

In [ ]:
y_train[y_train == 2] = 1
y_test[y_test == 2] = 1
y_val[y_val == 2] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.9495  | val_0_accuracy: 0.51902 |  0:00:17s
epoch 1  | loss: 0.858   | val_0_accuracy: 0.52413 |  0:00:34s
epoch 2  | loss: 0.80492 | val_0_accuracy: 0.52582 |  0:00:52s
epoch 3  | loss: 0.77714 | val_0_accuracy: 0.5331  |  0:01:09s
epoch 4  | loss: 0.75455 | val_0_accuracy: 0.53518 |  0:01:26s
epoch 5  | loss: 0.73813 | val_0_accuracy: 0.53665 |  0:01:44s
epoch 6  | loss: 0.72817 | val_0_accuracy: 0.54309 |  0:02:01s
epoch 7  | loss: 0.71797 | val_0_accuracy: 0.54451 |  0:02:19s
epoch 8  | loss: 0.71169 | val_0_accuracy: 0.55197 |  0:02:37s
epoch 9  | loss: 0.70596 | val_0_accuracy: 0.55206 |  0:02:55s
epoch 10 | loss: 0.70251 | val_0_accuracy: 0.55639 |  0:03:13s
epoch 11 | loss: 0.69765 | val_0_accuracy: 0.56136 |  0:03:31s
epoch 12 | loss: 0.69372 | val_0_accuracy: 0.56238 |  0:03:49s
epoch 13 | loss: 0.6912  | val_0_accuracy: 0.56669 |  0:04:06s
epoch 14 | loss: 0.68917 | val_0_accuracy: 0.5683  |  0:04:23s
epoch 15 | loss: 0.68735 | val_0_accuracy: 0.57033 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.7112
MCC: 0.4078


{'Accuracy': 0.7112391749989541,
 'Precision': 0.7407394228374194,
 'ROC AUC': np.float64(0.7757109544507503),
 'Recall': 0.76153719522233,
 'F1': 0.7509943450850943,
 'MCC': np.float64(0.4077502817040812)}

#CARVEME-BC-IV-100

In [ ]:
y_train[y_train == 1] = 0
y_train[y_train == 3] = 1
y_test[y_test == 1] = 0
y_test[y_test == 3] = 1
y_val[y_val == 1] = 0
y_val[y_val == 3] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.99805 | val_0_accuracy: 0.5087  |  0:00:08s
epoch 1  | loss: 0.93765 | val_0_accuracy: 0.50873 |  0:00:17s
epoch 2  | loss: 0.89245 | val_0_accuracy: 0.50856 |  0:00:25s
epoch 3  | loss: 0.85807 | val_0_accuracy: 0.5083  |  0:00:34s
epoch 4  | loss: 0.83101 | val_0_accuracy: 0.50879 |  0:00:43s
epoch 5  | loss: 0.81563 | val_0_accuracy: 0.50936 |  0:00:52s
epoch 6  | loss: 0.79426 | val_0_accuracy: 0.509   |  0:01:00s
epoch 7  | loss: 0.7812  | val_0_accuracy: 0.51317 |  0:01:09s
epoch 8  | loss: 0.7735  | val_0_accuracy: 0.5102  |  0:01:18s
epoch 9  | loss: 0.76348 | val_0_accuracy: 0.51045 |  0:01:27s
epoch 10 | loss: 0.75629 | val_0_accuracy: 0.51333 |  0:01:36s
epoch 11 | loss: 0.74994 | val_0_accuracy: 0.51218 |  0:01:45s
epoch 12 | loss: 0.74276 | val_0_accuracy: 0.51336 |  0:01:54s
epoch 13 | loss: 0.73862 | val_0_accuracy: 0.51517 |  0:02:03s
epoch 14 | loss: 0.73358 | val_0_accuracy: 0.51611 |  0:02:12s
epoch 15 | loss: 0.73051 | val_0_accuracy: 0.51344 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.6208
MCC: 0.2335


{'Accuracy': 0.6207927252825899,
 'Precision': 0.6026661260074003,
 'ROC AUC': np.float64(0.6540260405557718),
 'Recall': 0.538838031360464,
 'F1': 0.5689675798540496,
 'MCC': np.float64(0.23350629839964313)}

#CARVEME-BC-V-100

In [ ]:
y_train[y_train == 3] = 1
y_test[y_test == 3] = 1
y_val[y_val == 3] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.96154 | val_0_accuracy: 0.60767 |  0:00:11s
epoch 1  | loss: 0.86736 | val_0_accuracy: 0.60357 |  0:00:22s
epoch 2  | loss: 0.82879 | val_0_accuracy: 0.60773 |  0:00:34s
epoch 3  | loss: 0.79066 | val_0_accuracy: 0.61653 |  0:00:46s
epoch 4  | loss: 0.76454 | val_0_accuracy: 0.62289 |  0:00:57s
epoch 5  | loss: 0.74909 | val_0_accuracy: 0.62549 |  0:01:09s
epoch 6  | loss: 0.73478 | val_0_accuracy: 0.63008 |  0:01:21s
epoch 7  | loss: 0.72223 | val_0_accuracy: 0.63285 |  0:01:33s
epoch 8  | loss: 0.71336 | val_0_accuracy: 0.63444 |  0:01:44s
epoch 9  | loss: 0.70555 | val_0_accuracy: 0.63589 |  0:01:57s
epoch 10 | loss: 0.69839 | val_0_accuracy: 0.63714 |  0:02:08s
epoch 11 | loss: 0.69178 | val_0_accuracy: 0.63942 |  0:02:22s
epoch 12 | loss: 0.68707 | val_0_accuracy: 0.64072 |  0:02:34s
epoch 13 | loss: 0.68115 | val_0_accuracy: 0.64096 |  0:02:47s
epoch 14 | loss: 0.67817 | val_0_accuracy: 0.64385 |  0:02:59s
epoch 15 | loss: 0.67497 | val_0_accuracy: 0.64438 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.7130
MCC: 0.3254


{'Accuracy': 0.7130079627665469,
 'Precision': 0.6401146131805158,
 'ROC AUC': np.float64(0.7360442839188636),
 'Recall': 0.4054262510775373,
 'F1': 0.496430654704036,
 'MCC': np.float64(0.32540305275474873)}

#CARVEME-BC-I-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.96154 | val_0_accuracy: 0.60767 |  0:00:11s
epoch 1  | loss: 0.86736 | val_0_accuracy: 0.60357 |  0:00:23s
epoch 2  | loss: 0.82879 | val_0_accuracy: 0.60773 |  0:00:34s
epoch 3  | loss: 0.79066 | val_0_accuracy: 0.61653 |  0:00:46s
epoch 4  | loss: 0.76454 | val_0_accuracy: 0.62289 |  0:00:57s
epoch 5  | loss: 0.74909 | val_0_accuracy: 0.62549 |  0:01:09s
epoch 6  | loss: 0.73478 | val_0_accuracy: 0.63008 |  0:01:20s
epoch 7  | loss: 0.72223 | val_0_accuracy: 0.63285 |  0:01:32s
epoch 8  | loss: 0.71336 | val_0_accuracy: 0.63444 |  0:01:44s
epoch 9  | loss: 0.70555 | val_0_accuracy: 0.63589 |  0:01:55s
epoch 10 | loss: 0.69839 | val_0_accuracy: 0.63714 |  0:02:07s
epoch 11 | loss: 0.69178 | val_0_accuracy: 0.63942 |  0:02:18s
epoch 12 | loss: 0.68707 | val_0_accuracy: 0.64072 |  0:02:30s
epoch 13 | loss: 0.68115 | val_0_accuracy: 0.64096 |  0:02:41s
epoch 14 | loss: 0.67817 | val_0_accuracy: 0.64385 |  0:02:53s
epoch 15 | loss: 0.67497 | val_0_accuracy: 0.64438 |  0

#CARVEME-BC-II-50

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.68001 | val_0_accuracy: 0.83395 |  0:00:16s
epoch 1  | loss: 0.57172 | val_0_accuracy: 0.83198 |  0:00:32s
epoch 2  | loss: 0.54413 | val_0_accuracy: 0.83391 |  0:00:48s
epoch 3  | loss: 0.52489 | val_0_accuracy: 0.83414 |  0:01:03s
epoch 4  | loss: 0.50967 | val_0_accuracy: 0.83546 |  0:01:19s
epoch 5  | loss: 0.49833 | val_0_accuracy: 0.83603 |  0:01:34s
epoch 6  | loss: 0.48916 | val_0_accuracy: 0.8371  |  0:01:50s
epoch 7  | loss: 0.48271 | val_0_accuracy: 0.83752 |  0:02:06s
epoch 8  | loss: 0.4783  | val_0_accuracy: 0.83814 |  0:02:21s
epoch 9  | loss: 0.47291 | val_0_accuracy: 0.83875 |  0:02:37s
epoch 10 | loss: 0.46743 | val_0_accuracy: 0.83915 |  0:02:53s
epoch 11 | loss: 0.46401 | val_0_accuracy: 0.8389  |  0:03:08s
epoch 12 | loss: 0.46103 | val_0_accuracy: 0.83853 |  0:03:24s
epoch 13 | loss: 0.45892 | val_0_accuracy: 0.83876 |  0:03:40s
epoch 14 | loss: 0.45637 | val_0_accuracy: 0.83922 |  0:03:56s
epoch 15 | loss: 0.45408 | val_0_accuracy: 0.83962 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.8416
MCC: 0.0049


{'Accuracy': 0.8416181999293082,
 'Precision': 0.23529411764705882,
 'ROC AUC': np.float64(0.6362762996626697),
 'Recall': 0.0008130081300813008,
 'F1': 0.0016204172574437918,
 'MCC': np.float64(0.00494713999481246)}

#CARVEME-BC-III-50

In [ ]:
y_train[y_train == 2] = 1
y_test[y_test == 2] = 1
y_val[y_val == 2] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.7292  | val_0_accuracy: 0.7981  |  0:00:16s
epoch 1  | loss: 0.63599 | val_0_accuracy: 0.79428 |  0:00:33s
epoch 2  | loss: 0.60621 | val_0_accuracy: 0.79721 |  0:00:49s
epoch 3  | loss: 0.58232 | val_0_accuracy: 0.80051 |  0:01:06s
epoch 4  | loss: 0.56532 | val_0_accuracy: 0.80141 |  0:01:22s
epoch 5  | loss: 0.55435 | val_0_accuracy: 0.80278 |  0:01:39s
epoch 6  | loss: 0.54424 | val_0_accuracy: 0.80355 |  0:01:55s
epoch 7  | loss: 0.53647 | val_0_accuracy: 0.80411 |  0:02:12s
epoch 8  | loss: 0.52944 | val_0_accuracy: 0.8048  |  0:02:29s
epoch 9  | loss: 0.52515 | val_0_accuracy: 0.8048  |  0:02:45s
epoch 10 | loss: 0.52026 | val_0_accuracy: 0.80505 |  0:03:02s
epoch 11 | loss: 0.51699 | val_0_accuracy: 0.80553 |  0:03:18s
epoch 12 | loss: 0.51237 | val_0_accuracy: 0.80615 |  0:03:35s
epoch 13 | loss: 0.51075 | val_0_accuracy: 0.80635 |  0:03:52s
epoch 14 | loss: 0.50848 | val_0_accuracy: 0.80617 |  0:04:08s
epoch 15 | loss: 0.50537 | val_0_accuracy: 0.80645 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.8078
MCC: -0.0040


{'Accuracy': 0.807777663618617,
 'Precision': 0.8082894628609032,
 'ROC AUC': np.float64(0.6311238593302346),
 'Recall': 0.9991865268509692,
 'F1': 0.8936571724141851,
 'MCC': np.float64(-0.003961737604739982)}

#CARVEME-BC-IV-50

In [ ]:
y_train[y_train == 1] = 0
y_train[y_train == 3] = 1
y_test[y_test == 1] = 0
y_test[y_test == 3] = 1
y_val[y_val == 1] = 0
y_val[y_val == 3] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.94439 | val_0_accuracy: 0.62807 |  0:00:07s
epoch 1  | loss: 0.89526 | val_0_accuracy: 0.61933 |  0:00:14s
epoch 2  | loss: 0.85252 | val_0_accuracy: 0.61457 |  0:00:21s
epoch 3  | loss: 0.83113 | val_0_accuracy: 0.61755 |  0:00:28s
epoch 4  | loss: 0.80045 | val_0_accuracy: 0.6176  |  0:00:35s
epoch 5  | loss: 0.78466 | val_0_accuracy: 0.6253  |  0:00:42s
epoch 6  | loss: 0.76868 | val_0_accuracy: 0.62622 |  0:00:49s
epoch 7  | loss: 0.75378 | val_0_accuracy: 0.62712 |  0:00:56s
epoch 8  | loss: 0.74705 | val_0_accuracy: 0.63017 |  0:01:03s
epoch 9  | loss: 0.73606 | val_0_accuracy: 0.63157 |  0:01:10s
epoch 10 | loss: 0.7275  | val_0_accuracy: 0.63022 |  0:01:17s
epoch 11 | loss: 0.72118 | val_0_accuracy: 0.63531 |  0:01:24s
epoch 12 | loss: 0.71349 | val_0_accuracy: 0.63552 |  0:01:31s
epoch 13 | loss: 0.70854 | val_0_accuracy: 0.63756 |  0:01:38s
epoch 14 | loss: 0.70236 | val_0_accuracy: 0.63844 |  0:01:45s
epoch 15 | loss: 0.70043 | val_0_accuracy: 0.6396  |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.6478
MCC: 0.0073


{'Accuracy': 0.6478122928307605,
 'Precision': 0.649363952546572,
 'ROC AUC': np.float64(0.5573847906188387),
 'Recall': 0.9943096844792997,
 'F1': 0.7856412030031847,
 'MCC': np.float64(0.007299945426484224)}

#CARVEME-BC-V-50

In [ ]:
y_train[y_train == 3] = 1
y_test[y_test == 3] = 1
y_val[y_val == 3] = 1

In [ ]:
train_and_evaluate_tabnet(X_train, y_train, X_val, y_val, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.969   | val_0_accuracy: 0.56069 |  0:00:07s
epoch 1  | loss: 0.91816 | val_0_accuracy: 0.55402 |  0:00:15s
epoch 2  | loss: 0.87572 | val_0_accuracy: 0.5505  |  0:00:23s
epoch 3  | loss: 0.84542 | val_0_accuracy: 0.56223 |  0:00:30s
epoch 4  | loss: 0.8225  | val_0_accuracy: 0.5574  |  0:00:38s
epoch 5  | loss: 0.80191 | val_0_accuracy: 0.56208 |  0:00:46s
epoch 6  | loss: 0.7874  | val_0_accuracy: 0.56389 |  0:00:53s
epoch 7  | loss: 0.77405 | val_0_accuracy: 0.56614 |  0:01:01s
epoch 8  | loss: 0.76261 | val_0_accuracy: 0.56857 |  0:01:09s
epoch 9  | loss: 0.75636 | val_0_accuracy: 0.56772 |  0:01:17s
epoch 10 | loss: 0.74712 | val_0_accuracy: 0.57244 |  0:01:24s
epoch 11 | loss: 0.73959 | val_0_accuracy: 0.57588 |  0:01:32s
epoch 12 | loss: 0.73365 | val_0_accuracy: 0.57227 |  0:01:40s
epoch 13 | loss: 0.72856 | val_0_accuracy: 0.57714 |  0:01:47s
epoch 14 | loss: 0.72498 | val_0_accuracy: 0.57755 |  0:01:55s
epoch 15 | loss: 0.7192  | val_0_accuracy: 0.57935 |  0

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_results/tabnet_model.zip.zip

Test Metrics:
Accuracy: 0.6181
MCC: 0.1693


{'Accuracy': 0.6181200354955306,
 'Precision': 0.6433541898134612,
 'ROC AUC': np.float64(0.637881262859509),
 'Recall': 0.8016029143897996,
 'F1': 0.7138130149873484,
 'MCC': np.float64(0.1692667194503556)}